In [1]:
import heeps
from heeps.util.psf_template import psf_template
from heeps.util.lamD import get_lamD_mas
import numpy as np
from astropy.io import fits

### ELT aperture photometry

In [2]:
conf = dict(
    band='L',
    nframes = 1,
    add_phase = False,
)
conf = heeps.config.read_config(**conf)

In [3]:
conf.update(mode='ELT')
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf_OFF = heeps.wavefront.propagate(wf, onaxis=False, verbose=True, **conf)[0]
_, fwhm, ap_flux_ELT = psf_template(psf_OFF)
print('fwhm=%.2fpix, ap_flux=%.2e'%(fwhm, ap_flux_ELT))
print('( total flux =', np.sum(psf_OFF), ')')

Off-axis PSF:
‾‾‾‾‾‾‾‾‾‾‾‾‾
Create off-axis PSF cube
   2022-06-29 02:53:06, e2e simulation using 12 cores
   2022-06-29 02:53:07, completed in 0.91 seconds

fwhm=3.51pix, ap_flux=3.51e-01
( total flux = 0.9970582714582522 )


### RAVC

In [4]:
conf.update(mode='RAVC', ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf_OFF = heeps.wavefront.propagate_one(wf, onaxis=False, verbose=False, **conf)
_, fwhm, ap_flux = psf_template(psf_OFF)
print('fwhm=%.2fpix, ap_flux=%.2e'%(fwhm, ap_flux))
print('( total flux =', np.sum(psf_OFF), ')')
print('FWHM throughput =', ap_flux/ap_flux_ELT)
data = fits.getdata(conf['f_vc_trans'])
trans_vortex = np.interp(conf['lam']*1e6, data[0], data[1])
print('Vortex transmittance =', trans_vortex)
oat = fits.getdata(conf['dir_input'] + 'optics/oat_L_RAVC.fits')
npix = 5*get_lamD_mas(**conf)/conf['pscale']
oat5 = np.interp(npix, oat[0], oat[1])
print('OAT @ 5 lam/D =', oat5)
TOTAL =  ap_flux/ap_flux_ELT * trans_vortex * oat5
print('Total throughput =', TOTAL)

fwhm=3.30pix, ap_flux=6.78e-02
( total flux = 0.3648612634447917 )
FWHM throughput = 0.193080520944404
Vortex transmittance = 0.853226777335837
OAT @ 5 lam/D = 0.8828983317001562
Total throughput = 0.14544996960023243


### CVC

In [5]:
conf.update(mode='CVC', ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf_OFF = heeps.wavefront.propagate_one(wf, onaxis=False, verbose=False, **conf)
_, fwhm, ap_flux = psf_template(psf_OFF)
print('fwhm=%.2fpix, ap_flux=%.2e'%(fwhm, ap_flux))
print('( total flux =', np.sum(psf_OFF), ')')
print('FWHM throughput =', ap_flux/ap_flux_ELT)
data = fits.getdata(conf['f_vc_trans'])
trans_vortex = np.interp(conf['lam']*1e6, data[0], data[1])
print('Vortex transmittance =', trans_vortex)
oat = fits.getdata(conf['dir_input'] + 'optics/oat_L_CVC.fits')
npix = 5*get_lamD_mas(**conf)/conf['pscale']
oat5 = np.interp(npix, oat[0], oat[1])
print('OAT @ 5 lam/D =', oat5)
TOTAL =  ap_flux/ap_flux_ELT * trans_vortex * oat5
print('Total throughput =', TOTAL)

fwhm=3.48pix, ap_flux=2.20e-01
( total flux = 0.7938157033763246 )
FWHM throughput = 0.6257444133596661
Vortex transmittance = 0.853226777335837
OAT @ 5 lam/D = 0.8188439381099766
Total throughput = 0.4371823255551831


### CLC

In [6]:
conf.update(mode='CLC', ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf_OFF = heeps.wavefront.propagate_one(wf, onaxis=False, verbose=False, **conf)
_, fwhm, ap_flux = psf_template(psf_OFF)
print('fwhm=%.2fpix, ap_flux=%.2e'%(fwhm, ap_flux))
print('( total flux =', np.sum(psf_OFF), ')')
print('FWHM throughput =', ap_flux/ap_flux_ELT)
print('Total throughput =', ap_flux/ap_flux_ELT)

fwhm=3.85pix, ap_flux=1.68e-01
( total flux = 0.6309422107851245 )
FWHM throughput = 0.4793170095289134
Total throughput = 0.4793170095289134


### APP

In [7]:
conf.update(mode='APP')
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf_OFF = heeps.wavefront.propagate_one(wf, onaxis=False, verbose=False, **conf)
_, fwhm, ap_flux = psf_template(psf_OFF)
print('fwhm=%.2fpix, ap_flux=%.2e'%(fwhm, ap_flux))
print('( total flux =', np.sum(psf_OFF), ')')
print('FWHM throughput =', ap_flux/ap_flux_ELT)
data = fits.getdata(conf['f_app_trans'])
trans_app = np.interp(conf['lam']*1e6, data[0], data[1])
print('APP transmittance =', trans_app)
print('APP Strehl ratio =', conf['app_strehl'])
print('APP single PSF =', conf['app_single_psf'])
print('Total throughput =', ap_flux/ap_flux_ELT * conf['app_strehl'] * conf['app_single_psf'])

fwhm=3.56pix, ap_flux=2.36e-01
( total flux = 0.8061056187271394 )
FWHM throughput = 0.6705688636641192
APP transmittance = 0.7333889678395945
APP Strehl ratio = 0.64
APP single PSF = 0.48
Total throughput = 0.2059987549176174
